# Gathering seeing data

I had some discussions with [Miguel Rodríguez Alarcon (IAC)](https://scholar.google.com/citations?user=ySJtt44AAAAJ&hl=es&oi=ao) to discuss the best way to measure and keep track of seeing. 
* He discouraged me from using from using FWHM from images, as this parameter will combine astronomical seeing effects with instrumental seeing. 
* He suggested using the DIMM sensors available throughout the institute. 
* He suggested the idea of observing if variability in DIMM seeing is directly translated into higher FWHM in the images and, therefore, if FWHM can be used for strictly atmospheric seeing.

## Sources of DIMM seeing
* ING - WHTR2D2, accessible via <https://astro.ing.iac.es/seeing/r2d2_data.php>
* TNG - FGC-DIMM, I don't find access to raw data via web.

### Gathering data from R2D2

In [3]:
from datetime import datetime
import pandas as pd
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

from aspred.connector import r2d2_seeing

df = r2d2_seeing(datetime(2024, 1, 1), datetime(2024, 4, 1))

ImportError: cannot import name 'r2d2_seeing' from 'aspred.connector' (/Users/fabarzuza/Documents/GitHub/aspred/aspred/connector.py)

In [ ]:
df

In [ ]:
df

In [4]:
from urllib.request import urlopen

html = urlopen("https://astro.ing.iac.es/seeing/r2d2_data.php?date1=2024-01-01&date2=2024-05-09&submit=Submit")

In [5]:
text = html.read().decode()

In [6]:
text

'<pre>Date Time (UT)      R2D2 seeing (")<br>---------------------------------------<br>2024-01-01 00:00:31    1.1213<br>2024-01-01 00:01:06    1.0018<br>2024-01-01 00:01:41    0.7231<br>2024-01-01 00:02:16    0.8677<br>2024-01-01 00:02:52    0.8809<br>2024-01-01 00:03:27    0.9601<br>2024-01-01 00:04:02    0.8774<br>2024-01-01 00:04:37    0.8307<br>2024-01-01 00:05:14    0.9675<br>2024-01-01 00:05:49    0.8651<br>2024-01-01 00:06:25    0.8640<br>2024-01-01 00:07:00    0.8950<br>2024-01-01 00:07:35    0.8416<br>2024-01-01 00:08:11    0.7683<br>2024-01-01 00:08:46    0.8870<br>2024-01-01 00:09:21    0.9424<br>2024-01-01 00:09:57    0.7673<br>2024-01-01 00:10:32    0.9865<br>2024-01-01 00:11:07    0.8454<br>2024-01-01 00:11:43    0.7441<br>2024-01-01 00:12:19    0.8318<br>2024-01-01 00:12:55    0.8061<br>2024-01-01 00:13:30    0.7020<br>2024-01-01 00:14:05    0.6415<br>2024-01-01 00:14:41    0.7379<br>2024-01-01 00:15:16    0.8365<br>2024-01-01 00:15:51    0.8718<br>2024-01-01 00:16:26  

In [7]:
import requests
from bs4 import BeautifulSoup
import re

# URL of the webpage
url = "https://astro.ing.iac.es/seeing/r2d2_data.php?date1=2024-01-01&date2=2024-05-09&submit=Submit"

# Send a GET request to fetch the webpage content
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find and print the text content of the webpage preserving line breaks
    text_with_line_breaks = soup.get_text(separator='\n')

    # Replace multiple consecutive line breaks with a single one
    text_with_line_breaks = re.sub(r'\n+', '\n', text_with_line_breaks)
else:
    print("Failed to retrieve the webpage. Status code:", response.status_code)

text = text_with_line_breaks.split("\n")

dates = []
seeings = []

for i, line in enumerate(text):
    if i > 2:
        date = line[0:18]
        seeing = line[23:]
        dates.append(date)
        seeings.append(float(seeing))
    


In [8]:
dates = pd.to_datetime(pd.Series(dates), format="%Y-%m-%d %H:%M:%S")
seeings = pd.Series(seeings)

In [9]:
frame = {"Date": dates, "Seeing": seeings}

In [10]:
df = pd.DataFrame(frame)

In [11]:
df

,Date,Seeing
0,2024-01-01 00:01:00,1.0018
1,2024-01-01 00:01:04,0.7231
2,2024-01-01 00:02:01,0.8677
3,2024-01-01 00:02:05,0.8809
4,2024-01-01 00:03:02,0.9601
...,...,...
47737,2024-05-08 01:47:04,0.7054
47738,2024-05-08 01:48:03,0.7015
47739,2024-05-08 01:49:03,0.6485
47740,2024-05-08 01:50:02,0.7448


In [12]:
seeings

0        1.0018
1        0.7231
2        0.8677
3        0.8809
4        0.9601
          ...  
47737    0.7054
47738    0.7015
47739    0.6485
47740    0.7448
47741    0.8083
Length: 47742, dtype: float64